# MVP - Engenharia de Dados

Este MVP é uma análise de preço de imóveis. Visando entender quais características que podem valorizar um imóvel residencial.

**Coleta dos dados:**
A base de dados foi coletada do Kaggle - [House Prices Dataset](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)

**Questões a serem avaliadas:**

* Imóveis mais novos são mais caros?
* Qual bairro é o mais valorizado?
* Impacto da Qualidade do imóvel no Preço.

In [0]:
arq = "/Volumes/workspace/mvp_houses_price/mvp/train.csv"

# Leitura do CSV
df_houses = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load(arq)

# Salvando a tabela Bronze
df_houses.write.format("delta").mode("overwrite").saveAsTable("house_prices")

display(df_houses)

### Modelagem de Dados e Catálogo

Nesse processo da ETL iremos pegar as colunas necessárias para a realizar as análises. Transformando a área do porão que está como nulo para zero, e também será criado duas novas colunas:

1 - Idade Imóvel: A coluna com a idade que o imóvel possuí.

2 - Área Total: a área total do imóvel.



In [0]:
from pyspark.sql.functions import col, when, expr

df = spark.read.table("house_prices")

df_transform = df.select(
    col("Id"),
    col("Neighborhood").alias("Bairro"),
    col("YearBuilt").alias("AnoConstrucao"),
    col("GrLivArea").alias("Area"),
    col("SalePrice").alias("PrecoVenda"),
    col("YrSold").alias("AnoVenda"),
    col("OverallQual").alias("Qualidade"),
    col("GrLivArea").alias("AreaPorao")
)

# Tratamento dos nulos, transformar a Área do Porão em 0
df_transform = df_transform.fillna(0, "AreaPorao")


# Criando as novas colunas
df_house = df_transform \
    .withColumn("IdadeImovel", col("AnoVenda") - col("AnoConstrucao")) \
    .withColumn("AreaTotal", col("Area") + col("AreaPorao"))

# Carga
# Salvar tabela Silver para a análise
df_house.write.format("delta").mode("overwrite").saveAsTable("silver_house_prices")

print("ETL concluído!")
display(df_house)

### Análise da qualidade dos dados

Realizando a verificação de como está a qualidade dos dados a serenm utilizados, nos seguintes passos.

In [0]:
%sql
SELECT
    count(*) as total_imoveis,
    sum(CASE WHEN PrecoVenda <= 0 THEN 1 ELSE 0 END) as erro_preco_zerado,
    sum(CASE WHEN IdadeImovel < 0 THEN 1 ELSE 0 END) as erro_idade_negativa
FROM
    silver_house_prices

A análise de qualidade não encontoru nenhum registro inválido, como o preço ou idade negativas.

### Solução de problema 1

Imóveis mais novos são os mais caros?

In [0]:
%sql
SELECT 
    CASE 
        WHEN IdadeImovel <= 5 THEN '0-5 Anos - Novos'
        WHEN IdadeImovel <= 20 THEN '6-20 Anos - Seminovos'
        WHEN IdadeImovel <= 50 THEN '21-50 Anos - Antigos'
        ELSE '50+ Anos - Muito Antigos'
    END AS Faixa_Etaria,
    round(avg(PrecoVenda),2) AS Preco_Medio
FROM silver_house_prices
GROUP BY 1
ORDER BY Preco_Medio DESC

Databricks visualization. Run in Databricks to view.

Confirmou-se a hipótese de que imóveis mais novos tendem a ter maior valor de mercado, embora imóveis históricos (50+ anos) em boas condições apresentem resiliência de preço.


### Solução de Problema 2
Qual o Bairro mais valorizado?

In [0]:
%sql
SELECT
    Bairro,
    round(avg(PrecoVenda),2) as Preco_Medio,
    count(*) as Qtd_Vendas
FROM silver_house_prices
GROUP BY Bairro
HAVING count(*) > 10 
ORDER BY Preco_Medio DESC
LIMIT 10

Databricks visualization. Run in Databricks to view.

Com os resultados mostrados na tabela os dados mostram que o bairro de NoRidge é um bairro mais valorizado usando a métrica da média do valor das casas.



### Solução de Problemas 3
Impacto da qualidade do imóvel no preço.


In [0]:
%sql

SELECT
      Qualidade,
      round(avg(PrecoVenda),2) as Preco_Medio
FROM silver_house_prices
GROUP BY Qualidade
ORDER BY Preco_Medio ASC

Databricks visualization. Run in Databricks to view.

Acabamento: A variável Qualidade demonstrou um comportamento exponencial: melhorias na qualidade do acabamento resultam em aumentos desproporcionais no valor de venda.

### Conclusão e Considerações Finais
O presente projeto de MVP atingiu seu objetivo principal de construir um pipeline de dados para a análise de precificação de imóveis, utilizando o dataset House Prices. A solução demonstrou como dados brutos podem ser convertidos em inteligência de negócio através de processos estruturados de Engenharia de Dados.

#### Destaques Técnicos da Solução
A implementação técnica baseou-se nos pilares modernos de Data & Analytics, com destaque para:

Arquitetura em Camadas (Medallion Architecture): A separação lógica dos dados no Databricks provou-se eficiente.

A camada Bronze garantiu a ingestão segura e imutável do dado bruto (Raw).

A camada Silver foi o ponto focal de engenharia, onde utilizamos PySpark para tratamento de valores nulos (ex: fillna em áreas de porão) e Feature Engineering (criação das métricas IdadeImovel e AreaTotal).

Modelagem de Dados (Flat Table): A opção pelo modelo de Tabela Mestra (Flat/One Big Table) mostrou-se adequada à natureza do problema. Diferente de sistemas transacionais que exigem normalização, a abordagem analítica em Data Lake priorizou a leitura rápida e a simplificação de queries, eliminando a necessidade de joins complexos para este volume de dados, sem sacrificar a integridade da informação.

Qualidade de Dados: A implementação de checks de qualidade via SQL assegurou que distorções lógicas (como preços negativos ou datas de venda anteriores à construção) fossem identificadas, garantindo a confiabilidade das métricas finais.
